In [76]:
import pandas as pd
from shapely.geometry import  Point
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing as prepro
from sklearn.model_selection import train_test_split
import seaborn as sns
from matplotlib import cm
import urllib.request
import shutil
import zipfile
import os
import re
from matplotlib import pyplot as plt



In [21]:
pdir_data = pd.read_csv("..\project 4\Police_Department_Incident_Reports__2018_to_Present (1).csv")


In [23]:
# look at a few rows of the pdir_data file.
pdir_data.sample(5)

,Incident Datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Row ID,Incident ID,Incident Number,CAD Number,...,Longitude,Point,Neighborhoods,ESNCAG - Boundary File,Central Market/Tenderloin Boundary Polygon - Updated,Civic Center Harm Reduction Project Boundary,HSOC Zones as of 2018-06-05,Invest In Neighborhoods (IIN) Areas,Current Supervisor Districts,Current Police Districts
286447,2023/03/04 04:51:00 PM,2023/03/04,16:51,2023,Saturday,2023/03/06 11:53:00 AM,125129803401,1251298,230159724,230651341.0,...,-122.407853,POINT (-122.40785252141461 37.785491559268934),19.0,NaN,1.0,NaN,NaN,NaN,3.0,6.0
282874,2022/02/21 01:40:00 PM,2022/02/21,13:40,2022,Monday,2023/02/22 09:25:00 AM,124716668020,1247166,230127890,230530769.0,...,-122.430261,POINT (-122.43026080650851 37.77117368265581),28.0,NaN,NaN,NaN,NaN,NaN,11.0,7.0
318699,2018/02/26 11:00:00 PM,2018/02/26,23:00,2018,Monday,2018/02/27 10:34:00 AM,64032173000,640321,180154694,180581321.0,...,-122.476896,POINT (-122.4768956029359 37.74486944017707),40.0,NaN,NaN,NaN,NaN,NaN,7.0,10.0
116006,2022/06/29 01:02:00 PM,2022/06/29,13:02,2022,Wednesday,2022/06/29 01:34:00 PM,116565563010,1165655,220427042,221801431.0,...,-122.415883,POINT (-122.41588253952085 37.78351564008561),20.0,NaN,1.0,1.0,NaN,NaN,10.0,5.0
239532,2022/10/21 03:47:00 PM,2022/10/21,15:47,2022,Friday,2022/10/25 07:46:00 PM,120810906303,1208109,220735657,222982954.0,...,-122.423075,POINT (-122.42307492896889 37.76322118233464),37.0,NaN,NaN,NaN,NaN,NaN,5.0,3.0


In [25]:
# get the number of missing data points per column
missing_values_count = pdir_data.isnull().sum()

# look at the # of missing points in the first ten columns
missing_values_count[0:10]

Incident Datetime            0
Incident Date                0
Incident Time                0
Incident Year                0
Incident Day of Week         0
Report Datetime              0
Row ID                       0
Incident ID                  0
Incident Number              0
CAD Number              168715
dtype: int64

In [27]:
# how many total missing values do we have?
total_cells = np.product(pdir_data.shape)
total_missing = missing_values_count.sum()

# percent of data that is missing
(total_missing/total_cells) * 100

17.619319660017755

In [50]:
# remove all the rows that contain a missing value
pdir_data.dropna()

,Incident Datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Row ID,Incident ID,Incident Number,CAD Number,...,Longitude,Point,Neighborhoods,ESNCAG - Boundary File,Central Market/Tenderloin Boundary Polygon - Updated,Civic Center Harm Reduction Project Boundary,HSOC Zones as of 2018-06-05,Invest In Neighborhoods (IIN) Areas,Current Supervisor Districts,Current Police Districts


In [52]:
# remove all columns with at least one missing value
columns_with_na_dropped = pdir_data.dropna(axis=1)
columns_with_na_dropped.head()

,Incident Datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Row ID,Incident ID,Incident Number,Report Type Code,Report Type Description,Incident Code,Incident Description,Resolution,Police District
0,2023/03/13 11:41:00 PM,2023/03/13,23:41,2023,Monday,2023/03/13 11:41:00 PM,125373607041,1253736,230167874,VS,Vehicle Supplement,7041,"Vehicle, Recovered, Auto",Open or Active,Out of SF
1,2023/03/01 05:02:00 AM,2023/03/01,05:02,2023,Wednesday,2023/03/11 03:40:00 PM,125379506374,1253795,236046151,II,Coplogic Initial,6374,"Theft, Other Property, >$950",Open or Active,Mission
2,2023/03/13 01:16:00 PM,2023/03/13,13:16,2023,Monday,2023/03/13 01:17:00 PM,125357107041,1253571,220343896,VS,Vehicle Supplement,7041,"Vehicle, Recovered, Auto",Open or Active,Out of SF
3,2023/03/13 10:59:00 AM,2023/03/13,10:59,2023,Monday,2023/03/13 11:00:00 AM,125355107041,1253551,230174885,VS,Vehicle Supplement,7041,"Vehicle, Recovered, Auto",Open or Active,Out of SF
4,2023/03/14 06:44:00 PM,2023/03/14,18:44,2023,Tuesday,2023/03/14 06:45:00 PM,125402407041,1254024,230176728,VS,Vehicle Supplement,7041,"Vehicle, Recovered, Auto",Open or Active,Out of SF


In [62]:
# just how much data did we lose?
print("Columns in original dataset: %d \n" % pdir_data.shape[1])
print("Columns with na's dropped: %d" % columns_with_na_dropped.shape[1])

Columns in original dataset: 35 

Columns with na's dropped: 15


In [65]:
pdir_data.duplicated().sum()

0